In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
import tokamakTK
from tokamakTK import get_ECT_regression

import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches

from collections import Counter
from scipy.stats import f
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

pd.set_option('display.max_columns', None)
path = "../data/"
fig_path = "../../../LATEX/Latex Images/"

plt.rc('font',family = 'serif')

features = ['IP', 'BT', 'NEL', 'PLTH', 'RGEO', 'KAREA', 'EPS', 'MEFF']

In [3]:
# Obtained from Optimization

min_subset_ids_6357 = pd.read_csv(path+"R_ids_alpha_0.6357.csv")
min_subset_ids_9998 = pd.read_csv(path+"R_ids_alpha_0.9998.csv")

DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False) 

# Setting ELMy Dataset
DB5 = DB5[DB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]

# Removing Spherical TOKAMAKS
#DB5 = DB5[~DB5["TOK"].isin(['START','MAST','NSTX'])]

# There is two shots from DB2P8 missing in DB5
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

# Labeling shots that had great impact in decreasing alpha_R
DB5.insert(loc=2,column="label_6357",value=[0]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids_6357.id)].index), "label_6357"] = 1

DB5.insert(loc=2,column="label_9998",value=[0]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids_9998.id)].index), "label_9998"] = 1

DB5.insert(0, 'intercept', np.ones(len(DB5)))


print(
    "  Subset that decrease alpha-R to 0.6357\n--------\n" +
    f"{ round( (len(min_subset_ids_6357)/len(DB5))*100     ,2)  }% affected alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids_6357)/len(DB5))*100 ,2)  }% did not affect alpha_R" +
    "\n\n\n  Subset that decrease alpha-R to 0.9998\n--------\n" +
    f"{ round( (len(min_subset_ids_9998)/len(DB5))*100     ,2)  }% affected alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids_9998)/len(DB5))*100 ,2)  }% did not affect alpha_R"
)

  Subset that decrease alpha-R to 0.6357
--------
23.45% affected alpha_R
76.55% did not affect alpha_R


  Subset that decrease alpha-R to 0.9998
--------
9.88% affected alpha_R
90.12% did not affect alpha_R


In [4]:
# Data for alpha-R ~ 0.64

all_cases = [DB2.copy(),
             DB5.copy(),
             DB5[DB5.label_6357.isin([1]) | DB5.id.isin(DB2.id.values)],
             DB5[DB5.label_6357.isin([0])]
]

In [5]:
def get_FPvalues(full_model, reduced_model):
    return (anova_lm(reduced_model, full_model)["F"][1], 
            anova_lm(reduced_model, full_model)["Pr(>F)"][1])

In [6]:
# Creating table with F-test and P-test values per case
TESTS = pd.DataFrame(np.zeros((len(features), len(all_cases)*2)), 
                    columns=['FV0', 'PV0', 'FV1', 'PV1', 'FV2', 'PV2', 'FV3', 'PV3'],
                    index=features)

In [7]:
for i, data_ in enumerate(all_cases):
    for f in features:
        
        reduced_features = features.copy()
        reduced_features.remove(f)
        
        # get_ECT_regression takes the regression in log-space with intercept
        
        full_regression = get_ECT_regression(data_)
        reduced_regression = get_ECT_regression(data_, features=reduced_features)
        
        TESTS.loc[f, f"FV{i}"] = np.round(get_FPvalues(full_regression, reduced_regression)[0],5)
        TESTS.loc[f, f"PV{i}"] = np.round(get_FPvalues(full_regression, reduced_regression)[1],5)

In [8]:
TESTS

,FV0,PV0,FV1,PV1,FV2,PV2,FV3,PV3
IP,961.58446,0.0,4809.57120,0.00000,4165.11371,0.00000,2370.14462,0.0
BT,138.99369,0.0,25.79122,0.00000,1.01125,0.31469,300.46163,0.0
NEL,494.27457,0.0,332.97200,0.00000,85.68474,0.00000,1971.23255,0.0
PLTH,2674.96819,0.0,10024.60240,0.00000,2017.39751,0.00000,14937.76647,0.0
RGEO,2022.42722,0.0,3221.36266,0.00000,294.37717,0.00000,7450.69348,0.0
KAREA,94.66733,0.0,94.69404,0.00000,0.00198,0.96452,350.35744,0.0
EPS,131.13720,0.0,2.95960,0.08542,136.93138,0.00000,507.59434,0.0
MEFF,36.50700,0.0,157.27391,0.00000,25.50366,0.00000,182.72516,0.0
